# DA03_Function_Coulombic_Efficiency

This notebook demonstrates the execution of `DA03_Function_Coulombic_Efficiency.py`. We will go step by step through its functions, explaining their usage with examples.

## Coulombic Efficiency


### Calculate Coulombic Efficiency (CE)

In [ ]:

    ce_cycle = []
    cycle_columns = [col for col in df_VQ_grouped.columns if re.match(r'Cycle_\d+_', col)]
    cycle_numbers = sorted({int(re.search(r'Cycle_(\d+)_', col).group(1)) for col in cycle_columns})

    for cycle_id in cycle_numbers:
        plt.figure(figsize=(10, 6))
        
        cchg = df_VQ_grouped[f'Cycle_{cycle_id}_CapChg'].dropna()
        cdchg = df_VQ_grouped[f'Cycle_{cycle_id}_CapDChg'].dropna()
        total_CapChg = cchg.iloc[-1] if not cchg.empty else 0
        total_CapDChg = cdchg.iloc[-1] if not cdchg.empty else 0
        ce = (total_CapDChg / total_CapChg) * 100 if total_CapChg > 0 else 0


### Combine into a DataFrame & export result to csv file

In [ ]:
        ce_cycle.append({
            'Cycle_ID': cycle_id,
            'Discharge_Capacity': total_CapDChg,
            'Charge_Capacity': total_CapChg,
            'Coulombic_Efficiency': ce,
            })

    df_ce = pd.DataFrame(ce_cycle)
    df_ce.to_csv(f'{result_folder}/{file_name}/df_CE_{file_name}.csv', index=False)        
    print(df_ce)

### Plot CE to cycles

In [ ]:
    
    fig, ax1 = plt.subplots(figsize=(10, 6))

    color = 'tab:blue'
    ax1.set_xlabel('Cycle ID')
    ax1.set_ylabel('Coulombic Efficiency (%)', color=color)
    ax1.plot(df_ce['Cycle_ID'], df_ce['Coulombic_Efficiency'], color=color, 
             marker='o', label='Coulombic Efficiency')
    ax1.tick_params(axis='y', labelcolor=color)
    plt.ylim(0, 250)

    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Capacity (mAh)', color=color)
    ax2.plot(df_ce['Cycle_ID'], df_ce['Charge_Capacity'], color='tab:green', 
             marker='x', linestyle='-', label='Charge Capacity')
    ax2.plot(df_ce['Cycle_ID'], df_ce['Discharge_Capacity'], color=color, 
             marker='x', linestyle='--', label='Discharge Capacity')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    fig.legend(loc='upper left', bbox_to_anchor=(0.1,0.9))
    plt.title(f'Coulombic Efficiency and Capacity vs Cycle - {file_name}')
    plt.grid(True)
    plt.savefig(f'{result_folder}/{file_name}/CE-cycles_{file_name}.png', dpi=300)
    plt.close()
    
    return 
    
